<a href="https://colab.research.google.com/github/devyulbae/AIClass/blob/main/LLM_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 필수 설치 라이브러리

In [ ]:
!pip install -U langchain openai langchain-google-genai

In [ ]:
import os
from typing import Dict, List

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
# from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel

### API 키 입력

In [ ]:
import getpass
import os
# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["GOOGLE_API_KEY"] = getpass.getpass()


··········


### LLM 파트 구현
* 게임룰에 대한 정보들을 얻는 방법을 프롬프트 체인을 이용해 구성했습니다.
* 부루마블이라는 보드게임을 진행하기위한 기본적인 rule과 건물을 지을 수 있는 규칙이 들어간 데이터를 이용해서 문답을 진행합니다.

In [ ]:
RULE_1 = """칸트

칸트의 사상 체계는 흔히 크게 세 갈래로 나눠진다:
	•	인식론: "나는 무엇을 알 수 있는가?"
	•	윤리학: "나는 무엇을 해야 하는가?"
	•	종교철학: "나는 무엇을 희망해도 좋은가?"

이 중 첫째 질문인 "나는 무엇을 알 수 있는가?"는 『순수 이성 비판』에서 주로 다루어진다. 첫째 질문은 자신이 문제삼는 것에서 분명하게 드러나듯, 인간 이성이 인식할 수 있는 범위와 한계가 어디까지인지에 대한 물음이다. 이는 다시 말해, 우리는 어디까지 알 수 있으며, 또한 어떤 것은 알 수 없는지를 논의하겠다는 것을 의미한다. 이러한 점에서 『순수 이성 비판』은 첫째 질문을 본격적으로 논의면서, 인간 이성의 능력이 지닌 범위와 한계가 어디까지인지를 적극적으로 검토한다. 이 점에서 『순수 이성 비판』은 인간의 인식과 앎에 대한 논의를 첫째 질문을 통해 수행하고 있음을 알 수 있다. 칸트에 따르면, 이성주의(합리주의) 철학의 전통에서 이성은 그 능력이 검증되지 않은 채, 신, 영혼불멸, 자유와 같은 무제약자들을 함부로 추구했다는 문제가 있다. 그러나 이러한 신과 같은 무제약자들은 인간 이성이 인식할 수 있는 한계를 아득히 넘어서는 초감성적인 대상들로, 유한한 인간 이성은 자신의 한계로 인해 결코 인식할 수 없다. 따라서 인간 이성이 인식할 수 있는 범위는 오직 자연 인과성의 지배를 받는 감성적인 경험적 대상들밖에 없으며, 이러한 범위를 넘어서서 초감성적인 대상들을 인식하려는 모든 시도들은 독단적인dogmatisch 것일 수밖에 없다.

둘째 질문인 "나는 무엇을 해야만 하는가?"는 도덕적 물음으로, 『도덕형이상학 정초』와 『실천 이성 비판』에서 다루어진다. 둘째 물음에 대한 답변, 즉 우리가 해야만 하는 일은 결국 도덕적 행동이다. 우리는 도덕적으로 행동해야만 한다. 그렇다면 도덕적 행동은 무엇인가? 바로 도덕법칙에 따르고자 하는 행위 원리를 받아들여 행동하는 것이다. 이처럼 도덕법칙에 따라 행동하는 것의 의미가 무엇인지에 대해 칸트는 『도덕형이상학 정초』와 『실천 이성 비판』 전반부 「순수 실천이성의 분석론」에서 비판적으로 검토하고 있다.

마지막으로 셋째 물음인 "나는 무엇을 희망해도 좋은가?"는 칸트의 여러 저서에서 복합적으로 논의되고 있다. 칸트의 저서에서 이 셋째 물음이 최초로 등장한 저서는 『순수 이성 비판』으로, 칸트는 『순수 이성 비판』의 후반부인 「방법론」 중에서도 「순수 실천이성의 규준」 장에서 이 물음의 의미를 본격적으로 제시하고 있다.[79] 『순수 이성 비판』에 따르면, 우리가 둘째 물음에 따라 도덕성을 확보하는 일은 우리가 "행복해도 좋을 자격"을 갖추는 일이다. 따라서 우리가 충분히 도덕적 행위 원리를 받아들여 행복해도 좋을 자격을 가지게 된다면, 비로소 우리는 우리의 도덕성에 상응하는 정도만큼 행복해질 수 있을 것이라고 희망할 수 있게 된다는 것이다. 다만 이 행복이 우리에게 실제로 주어진다고 보지 않고, 희망의 대상이라고 말한다는 점에서 우리는 칸트는 전통적인 행복주의 윤리학을 적극적으로 거부하였음을 알 수 있는데, 왜냐하면 전통적인 행복주의 윤리학은 도덕적으로 행동하게 될 경우 현세의 삶에서 도덕적 행복을 얻을 수 있다고 주장하기 때문이다. 즉 행복주의와는 달리 칸트는 우리가 아무리 도덕적으로 살아간다고 하더라도, 자연법칙이 지배하는 감성계에서는 결코 도덕성에 상응하는 행복이 도출될 수 없으며, 따라서 우리가 도덕적으로 산다 해도 얻을 수 있는 것은 단지 행복해도 좋을 자격, 그리고 그 자격에 행복이 따라오리라는 희망 뿐이라고 주장한다. 또한 칸트에 따르면, 도덕성에 상응하는 행복은 결코 우리 인간의 능력으로는 실현 불가능한 것이지만, 행복의 분배에 개입하는 전능한 신을 가정한다면 각자의 도덕성에 상응하는 정도만큼 그 행복을 희망할 수 있게 되므로, 이러한 도덕에 상응하는 행복이 보장될 수 있게 하기 위해서는 신의 현존을 요청해야만 한다. 이 점에서 "나는 무엇을 희망해도 좋은가?"라는 물음은 신의 현존을 요청하는 요청이론Postulatslehre으로 이행하게 되며, 이는 칸트의 고유한 종교철학 이론인 도덕신학Moralstheologie을 구성하는 주요한 체계가 된다. 그러므로 셋째 물음은 종교적 물음과 필연적으로 연관될 수밖에 없다.

이처럼 『순수 이성 비판』에서 제시된 "나는 무엇을 희망해도 좋은가?"라는 물음은 결국 행복할 자격과 행복이 필연적으로 연결되는 최고선이라는 도덕적 이상의 문제로 연결된다. 그러므로 이 세 번째 물음은 최고선의 문제를 다룬 여러 저서들에서 간접적으로, 또 반복적으로 등장한다고 이해할 수 있다. 이는 『실천 이성 비판』의 후반부인 「변증론」에서는 영혼불멸과 신의 현존의 요청을 통해 최고선의 실현에 대한 희망으로 나타나며, 『이성의 한계 안에서의 종교』에서는 근본악의 문제와 그 원인인 악의 원리에 대항하는 윤리적 공동체구현 내지는 최고선에 대한 희망으로, 또 『판단력비판』에서는 자연과 자유의 통일을 통해 이 지상에서 실현될 수 있을 자연의 궁극목적이자 목적들의 나라인 최고선에 대한 희망으로 나타나기도 한다.

[message]
{input}

Answer:"""

In [ ]:
RULE_2 = """데카르트

데카르트는 철학적 논쟁을 "무엇이 참된가"에서 "내가 무엇을 확신할 수 있는가"로 전환시켰고, 이를 통해 진리의 권위를 보장하는 역할을 신에서 인간으로 바꾸었다.[42] "무엇이 참된가"에서는 진리의 근거로 신을 말할 수 있지만, "내가 무엇을 확신할 수 있는가"는 전적으로 개인의 판단에 달려있어 '진리의 근거로서의 신'은 부차적인 문제가 되어버리기 때문이다.

즉, 데카르트 이후로 '신 중심'의 철학은 점차 '인간 이성 중심'의 철학으로 바뀌어 갔다. 진리의 보증인은 더 이상 신이 아니라 인간이며, 각자는 자신의 현실에 대한 '자의식의 형성자'로서 '생각하는 주체'를 가진다. 주체를 가진 각 사람은 하나님께 순종하는 아이가 아니라 스스로의 이성을 가지고 행동하는 어른이 된다. 이제 인류는 기독교 계시 진리와 교회 교리로부터 해방되어, 스스로 법을 만들고 자기 입장을 취할 수 있게 되었다.[43] 이때문에 데카르트의 철학적 작업은 기독교 중세에서 근대 주체로의 전환을 극적으로 성취한 혁명적인 결과라고 평가받으며, 지금도 그 영향이 이어지고 있는 것이다. 인간 이성을 자율적으로 확립한 그의 관점은 이후 계몽주의가 신과 교회로부터 해방되는 기초를 제공한다.[44][45][46]

물론 그렇다고 해서 데카르트가, 이전 시대의 철학사조와는 전혀 상관이 없이 어느날 갑자기 비연속적으로 출현한 것은 아니다. 가령 데카르트의 신 존재 증명은 그 자체로는 딱히 발칙한 시도라거나 반골적인 시도가 아니였으며, 오히려 스콜라학자 안셀무스가 보인 신 존재 증명의[47] 전통에 연장선이라 할 수 있다. 데카르트는 신 존재 증명에 한에서는 명백히 스콜라적 전통에 깊이 뿌리박고 있었으며,[48] 아우구스티누스, 안셀무스, 둔스 스코투스, 토마스 아퀴나스의 노선에서, 이성으로 신 존재를 증명하려고 한 후기 스콜라학자 중 한 명이라고 말할 수 있을 것이다.[49] 그러나 그의 신 존재 증명은 앞선 학자들과 "거의 동일한 내용"을 말했다는 점에서, 이는 결코 업적이라 볼 수 없으며 데카르트의 한계라고 말해진다.[50] 데카르트가 철학계에서 "근대 철학의 아버지"라 불리며 높이 평가받는 부분은 그가 '코기토'를 발견했다는 것에 있지, 신존재를 증명한 것에 있었던 것은 아니었기 때문이다.

데카르트가 기존 스콜라학자들과 구분되는 지점은, 그가 철학에서 우선 순위를 '신' 보다 '인간의 이성'에 먼저 두었다는 점에 있다. 즉, 데카르트는 "철학의 제1원리"를 '코기토(생각하는 주체)'에 두었다는 점에서, 기존 스콜라학자들과 전적으로 다른 길을 걸어갔다. 기존 스콜라학자들도 '이성'으로 신존재를 증명하려고 했던 것은 맞으나, 그들에게 있어서는 무엇보다도 '신'이 철학의 제1원리였고, 인간의 이성은 단지 '신 존재 또는 신이 만든 세계'를 이해하기 위한 인간의 능력일 뿐이었다.[51] 반면 데카르트는 그 순서를 뒤집어 "생각하는 주체"인 인간의 이성이 철학의 첫번째 원리가 되어야 한다고 주장했고, 이를 통해 "근대철학"의 포문을 열었던 것이다.
데카르트는 새로운 것의 대변자인 동시에 낡은 것의 대표자이기도 하였다. 그는 모든 것을 일소하고 철학을 새롭고 확실한 토대 위에 정초하고자 하였으나 동시에 그의 사상은 무엇보다 그의 신 존재 증명에서 볼 수 있듯이 스콜라적 전통에 깊이 뿌리박고 있었다.

Gunnar Skirbekk · Nils Gilje [52]
그렇기에 데카르트는 보통 근대 철학자로 분류됨에도 불구하고, (신 존재 증명에 한해서) 후기 스콜라학자로 분류하는 것도 가능하다. 따라서 이렇게 정리할 수 있을 것이다. 데카르트의 신 존재 증명은 스콜라적 전통에 깊이 뿌리박혀 있고 그것은 낡은 것을 대표하는 것이지만, 철학과 신을 분리하는 단초를 제공했다는 점에서 데카르트는 새로운 것의 대변자이자 그 낡은 모든 것을 일소하여 철학을 새롭고 확실한 토대 위에 정초하고자 한 인물이었다.

[message]
{input}

Answer:"""

In [ ]:
RULE_3 = """공자

사상적 측면에서는 동아시아적 휴머니즘이라고 할 수 있는 '인(仁)' 을 최초로 제시하였다.[21] 인(仁)이란, 도덕적, 인본주의적, 인문주의적인 의미의 '사람다움', 즉, '사람의 따뜻한 마음'을 말한다.

그러면 "사람다움"이란 무엇일까? 이는 공자(《논어》)의 仁者 愛人('인'은 남을 아끼는 것이다)에서 알 수 있다. 이는 우리네 어머니들이 자식을 대할 때 그러하듯이, 배려하고 걱정하는 등, 조건없이 남을 챙기고 아끼는 마음이 "사람다움"의 본질임을 말해주고 있다. 그 당시 수많은 사람들이 전쟁과 배신 속에서 죽었기에 '남을 챙기고 아끼자. 그래서 사람답게 살자.'는 공자의 외침은 당시 사회를 안정시키고자 하는 수많은 사상가들에게 크고 작은 영감을 주었다.

이렇게 주변을 아끼고 챙기는 사람다움(仁)을 잘하는 사람은 필연적으로 사람들의 신뢰(信)를 얻게 되는데, 주변의 수많은 신뢰를 통해 높혀진 그 사람은 자신이 속한 단체의 장(長)이 된다. 이로써 윗사람과 아랫사람이 생기게 되니, 윗사람은 아랫사람의 모범이 되어 가르치고, 아랫사람은 윗사람을 본받아 배운다. 공자는 이러한 상하관계에서의 올바름을 '의'(義)라고 하여, 공정하고 의로운 자가 높은 위치에 있어야 그 사회가 안정이 된다는 것을 강조하였다. 또한 주변 친한 사람을 챙기는 것도 너무 지나치면 사람을 피곤하게 만들고, 상하관계의 가르치고 배우는 것에 있어서 순서가 없으면 미숙한 아랫사람이 그 미숙함으로 윗사람을 우습게 보아 가르침이 제대로 전해지지 않으니, 친하더라도 거리를 지키게 하고 상하관계에서는 순서를 따르게 하였는데, 이러한 형식을 '예'(禮)라고 한 것이다. 따라서 인(仁)한 사람은 결국 높은 자리로 나아가 의로움(義)을 펼치며, 이러한 인(仁)과 의(義)의 적절한 순서를 지키자는 것이 예(禮)가 되는 셈. 여기서부터 효제자(孝悌慈), 서(恕), 경(敬)의 윤리관, 예치(禮治), 덕치(德治)의 정치관 등이 만들어졌다.

따라서 공자는 세상이 혼란한 연유를 이러한 인(仁)의 부재와 예악(禮樂)의 상실에서 찾았으며, 예악을 따르는 인군(仁君)을 일으켜 주(周)나라 초기와 같은 성세(聖世)를 회복하는 것을 이상으로 삼았다. 정치는 인(仁)을 갖추고 예(禮)에 밝은 군자(君子)[23]가 주도해야 한다고 보았고, 이 군자를 양성함에 신분을 가리지 않고 보편적 교육을 베풀었다. 그가 정립(正立)한 사상, 그리고 그의 사후 후대 학자들이 그 사상을 뼈대로 하고 여러 사상을 곁들여[24][25] 발전시켜온 일련의 사상계(思想界)를 우리는 유학(儒學), 또는 유교(儒敎)라고 부른다.

오늘날 세간에서는 보통 공자를 '유교의 창시자' 라고 인식하고 있지만, 적어도 공자는 자신이 무언가의 창시자라는 생각을 한 적이 없었다. 공자는 다만 스스로 옛날부터 이어져 오던 예악(禮樂) 전통을 정리하고 계승했을 따름이며[26], 자신의 사상은 새로운 것이 아니고 옛 성왕(聖王), 성현(聖賢)의 행적에 깃든 참뜻을 밝혀낸 것일 뿐이라고 하였다. 유명한 술이부작(述而不作), 즉 "저술한 것이지 창작한 것은 아니다."는 말은 이것을 뜻하는 것이다. 그의 목표는 요순우탕문무주공의 사상을 잇는 것이었다. 실제로, 흔히 유교 문화로 알려진 것들 중에는 공자 이전부터 있었거나 혹은 나중에 가서야 형성된 관습들이 많다. 또한, 공가(孔家)라고 부르지 않고, 유가(儒家)라고 부르는 것도 후대의 학자들이 이런 점을 감안했기 때문이다.

원 성종 11년(대덕大德 11년, 1307년)에 '대성지성문선왕'(大成至聖文宣王)이란 시호를 내렸다. 이것이 현재 성균관 대성전 등지의 공문사당 위패에 표기되는 공식 존호이기도 하다.[27] 동아시아 문화권에서 누군가를 죽은 뒤 추존하는 사례는 많지만 대부분 왕족을 정통성 때문에 추존했던 것이지, 공자처럼 왕족도 아니었던 사람을 군주로 추존하는 사례는 매우 드물었다.[28]

보통 《논어》에서는 '자왈(子曰: 선생님께서 말씀하시기를)'이라고 줄여서 말한다. 《논어》 이외의 책에서 공자를 지칭할 때는 '부자(夫子)'라는 표현을 쓰기도 하는데 선생님이라는 뜻이다. 이런 까닭은 한자문화권에서 선생님이라고 불리는 사람은 보통 공자를 뜻하기 때문이다. 간혹 '공부자(孔夫子)'라고 표현하기도 하는데, 서구 선교사들은 이 표현을 라틴어로 바꿔서 '콘푸치우스(Confucius)'라고 불렀다.[29] '공부자'의 중국어 발음[30]에다가 남성 명사 주격 단수 어미인 '-us'를 붙인 것.[31]

[message]
{input}

Answer:"""

In [ ]:
def create_chain(llm, template_prompt, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=template_prompt
        ),
        output_key=output_key,
        verbose=True,
    )


llm = ChatGoogleGenerativeAI(model="gemini-pro")

rule_1 = create_chain(
    llm=llm,
    template_prompt=RULE_1,
    output_key="text",
)
rule_2 = create_chain(
    llm=llm,
    template_prompt=RULE_2,
    output_key="text",
)
rule_3 = create_chain(
    llm=llm,
    template_prompt=RULE_3,
    output_key="text",
)


destinations = [
    "칸트: 칸트에 관련된 이야기는 이 키워드를 선택해줘",
    "데카르트: 데카르트에 관련된 이야기는 이 키워드를 선택해줘",
    "공자: 공자에 관련된 이야기는 이 키워드를 선택해줘"
]
destinations = "\n".join(destinations)
router_prompt_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
router_prompt = PromptTemplate.from_template(
    template=router_prompt_template, output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True)

multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains={
        "칸트": rule_1,
        "데카르트": rule_2,
        "공자": rule_3,
    },
    default_chain=ConversationChain(llm=llm, output_key="text"),
)


class UserRequest(BaseModel):
    user_message: str


def gernerate_answer(req: UserRequest) -> Dict[str, str]:
    context = req.dict()
    context["input"] = context["user_message"]
    answer = multi_prompt_chain.invoke(context)

    return {"answer": answer}

### User 데이터 입력
* 유저 데이터 입력 후 결과를 확인 합니다.

In [ ]:
user_data = {
    "user_message": "칸트는 어떤 철학을 가지고 있어?"
}


In [ ]:
request_instance = UserRequest(**user_data)

In [ ]:
gernerate_answer(request_instance)



> Entering new LLMRouterChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 칸트 

칸트의 사상 체계는 흔히 크게 세 갈래로 나눠진다:
	•	인식론: "나는 무엇을 알 수 있는가?"
	•	윤리학: "나는 무엇을 해야 하는가?"
	•	종교철학: "나는 무엇을 희망해도 좋은가?"

이 중 첫째 질문인 "나는 무엇을 알 수 있는가?"는 『순수 이성 비판』에서 주로 다루어진다. 첫째 질문은 자신이 문제삼는 것에서 분명하게 드러나듯, 인간 이성이 인식할 수 있는 범위와 한계가 어디까지인지에 대한 물음이다. 이는 다시 말해, 우리는 어디까지 알 수 있으며, 또한 어떤 것은 알 수 없는지를 논의하겠다는 것을 의미한다. 이러한 점에서 『순수 이성 비판』은 첫째 질문을 본격적으로 논의면서, 인간 이성의 능력이 지닌 범위와 한계가 어디까지인지를 적극적으로 검토한다. 이 점에서 『순수 이성 비판』은 인간의 인식과 앎에 대한 논의를 첫째 질문을 통해 수행하고 있음을 알 수 있다. 칸트에 따르면, 이성주의(합리주의) 철학의 전통에서 이성은 그 능력이 검증되지 않은 채, 신, 영혼불멸, 자유와 같은 무제약자들을 함부로 추구했다는 문제가 있다. 그러나 이러한 신과 같은 무제약자들은 인간 이성이 인식할 수 있는 한계를 아득히 넘어서는 초감성적인 대상들로, 유한한 인간 이성은 자신의 한계로 인해 결코 인식할 수 없다. 따라서 인간 이성이 인식할 수 있는 범위는 오직 자연 인과성의 지배를 받는 감성적인 경험적 대상들밖에 없으며, 이러한 범위를 넘어서서 초감성적인 대상들을 인식하려는 모든 시도들은 독단적인dogmatisch 것일 수밖에 없다.

둘째 질문인 "나는 무엇을 해야만 하는가?"는 도덕적 물음으로, 『도덕형이상학 정초』와 『실천 이성 비판』에서 다루어진다. 둘째 물음에 대한 답변, 즉 우리가 해야만 하는 일은 결국 도덕적 행동이다. 우리는 도덕적으로 행동해야만 한다. 그렇다면 도덕적 행동은 무엇인가? 바로 

{'answer': {'user_message': '칸트는 어떤 철학을 가지고 있어?',
  'input': '칸트는 어떤 철학을 가지고 있어?',
  'text': '칸트는 인식론, 윤리학, 종교철학으로 나뉘는 철학 체계를 가지고 있습니다. \n\n* 인식론: "나는 무엇을 알 수 있는가?"\n이 질문은 인간 이성의 능력이 지닌 범위와 한계가 어디까지인지를 적극적으로 검토합니다. 칸트에 따르면, 이성은 그 능력이 검증되지 않은 채, 신, 영혼불멸, 자유와 같은 무제약자들을 함부로 추구했고, 이러한 무제약자들은 인간 이성이 인식할 수 있는 한계를 아득히 넘어서는 대상들이며, 따라서 인간 이성이 인식할 수 있는 범위는 감성적인 경험적 대상들 뿐입니다. \n\n* 윤리학: "나는 무엇을 해야 하는가?"\n이 질문은 우리가 해야만 하는 일은 도덕적 행동이라고 답변하며, 도덕적 행동은 도덕법칙에 따라 행동하는 것입니다. 도덕법칙은 우리가 행동할 때마다 따라야 할 절대적인 원칙이며, 칸트는 이러한 법칙을 선험적인 것으로 생각했습니다.\n\n* 종교철학: "나는 무엇을 희망해도 좋은가?"\n이 질문은 도덕성에 상응하는 행복이 보장될 수 있게 하기 위해서는 신의 현존을 요청해야만 한다고 주장합니다. 따라서 종교철학은 신의 현존을 요청하는 요청이론으로 이행하게 되며, 이는 칸트의 고유한 종교철학 이론인 도덕신학을 구성하는 주요한 체계가 됩니다.'}}